In [21]:
from utils.data import load_data
import torch
import torch.nn as nn
from model.mscred import *

dataloader = load_data()
for x in dataloader["train"]:
    input = x
    break
input = input[0]

class CnnEncoder(nn.Module):
    def __init__(self, in_channels_encoder):
        super(CnnEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels_encoder, 32, 3, 1, 1),
            nn.SELU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.SELU()
        )    
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 2, 2, 1),
            nn.SELU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 2, 2, 0),
            nn.SELU()
        )
    def forward(self, X):
        conv1_out = self.conv1(X)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        conv4_out = self.conv4(conv3_out)
        return conv1_out, conv2_out, conv3_out, conv4_out


cnnEncoder = CnnEncoder(3)
conv1_out, conv2_out, conv3_out, conv4_out = cnnEncoder(input)
conv1_out.shape

torch.Size([5, 32, 30, 30])

In [46]:
def attention(ConvLstm_out):
    attention_w = []
    for k in range(5):
        attention_w.append(torch.sum(torch.mul(ConvLstm_out[k], ConvLstm_out[-1]))/5)
    m = nn.Softmax(dim=1)
    attention_w = torch.reshape(m(torch.stack(attention_w)), (-1, 5))
    cl_out_shape = ConvLstm_out.shape
    ConvLstm_out = torch.reshape(ConvLstm_out, (5, -1))
    convLstmOut = torch.matmul(attention_w, ConvLstm_out)
    convLstmOut = torch.reshape(convLstmOut, (1, cl_out_shape[1], cl_out_shape[2], cl_out_shape[3]))
    return convLstmOut

class Conv_LSTM(nn.Module):
    def __init__(self):
        super(Conv_LSTM, self).__init__()
        self.conv1_lstm = ConvLSTM(input_channels=32, hidden_channels=[32], 
                                   kernel_size=3, step=5, effective_step=[4])
        self.conv2_lstm = ConvLSTM(input_channels=64, hidden_channels=[64], 
                                   kernel_size=3, step=5, effective_step=[4])
        self.conv3_lstm = ConvLSTM(input_channels=128, hidden_channels=[128], 
                                   kernel_size=3, step=5, effective_step=[4])
        self.conv4_lstm = ConvLSTM(input_channels=256, hidden_channels=[256], 
                                   kernel_size=3, step=5, effective_step=[4])


    def forward(self, conv1_out, conv2_out, 
                conv3_out, conv4_out):
        conv1_lstm_out = self.conv1_lstm(conv1_out)
        print(conv1_lstm_out[0][0].shape)
        conv1_lstm_out = attention(conv1_lstm_out[0][0])
        conv2_lstm_out = self.conv2_lstm(conv2_out)
        conv2_lstm_out = attention(conv2_lstm_out[0][0])
        conv3_lstm_out = self.conv3_lstm(conv3_out)
        conv3_lstm_out = attention(conv3_lstm_out[0][0])
        conv4_lstm_out = self.conv4_lstm(conv4_out)
        conv4_lstm_out = attention(conv4_lstm_out[0][0])
        return conv1_lstm_out.unsqueeze(0), conv2_lstm_out.unsqueeze(0), conv3_lstm_out.unsqueeze(0), conv4_lstm_out.unsqueeze(0)
conv_LSTM = Conv_LSTM()

conv1_lstm_out, conv2_lstm_out, conv3_lstm_out, conv4_lstm_out = conv_LSTM(
                                conv1_out, conv2_out, conv3_out, conv4_out)

torch.Size([5, 32, 30, 30])
torch.Size([32, 30, 30])
torch.Size([64, 15, 15])
torch.Size([128, 8, 8])
torch.Size([256, 4, 4])


In [36]:
a = torch.tensor([[1, 2]])
b = torch.tensor([[1, 2], [1, 2]])
torch.mul(a, b)

tensor([[1, 4],
        [1, 4]])

In [37]:
a = torch.randn([5, 32, 30, 30])

In [40]:
conv_shape = a.shape[0]
conv_shape

5

In [7]:
a

tensor([[ 0.6345, -0.9279,  0.9439],
        [-0.0653, -0.7846, -0.5672]])tensor([[-0.6386, -0.3668, -0.1110],
        [ 1.5943,  0.4609, -1.0188]])
tensor([[-0.4051,  0.3403, -0.1048],
        [-0.1042, -0.3616,  0.5779]])
